In [106]:
import PyPDF2 
import re
import random
import torch 
import numpy as np 
import spacy
import string
from tqdm import tqdm
from torchtext.data.utils import get_tokenizer
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

## DATA PREPROCESSING

In [107]:
with open("en_US.blogs.txt","r") as f: 
    text = f.read() 

In [108]:
print('Total length of the input (charachters in the text): ',len(text))
print('initial estimation of number of words: ',len(text.split()))
print('initial estimation of vocabulary:',len(set(text.split())))


Total length of the input (charachters in the text):  207723793
initial estimation of number of words:  37334131
initial estimation of vocabulary: 1103548


In [109]:
print(text[:1000])

In the years thereafter, most of the Oil fields and platforms were named after pagan “gods”.
We love you Mr. Brown.
Chad has been awesome with the kids and holding down the fort while I work later than usual! The kids have been busy together playing Skylander on the XBox together, after Kyan cashed in his $$$ from his piggy bank. He wanted that game so bad and used his gift card from his birthday he has been saving and the money to get it (he never taps into that thing either, that is how we know he wanted it so bad). We made him count all of his money to make sure that he had enough! It was very cute to watch his reaction when he realized he did! He also does a very good job of letting Lola feel like she is playing too, by letting her switch out the characters! She loves it almost as much as him.
so anyways, i am going to share some home decor inspiration that i have been storing in my folder on the puter. i have all these amazing images stored away ready to come to life when we get o

In [110]:
text_sentences = text.split('\n')
print('total sentences in the text',len(text_sentences))
print('total unique sentences in the text',len(set(text_sentences)))

total sentences in the text 899289
total unique sentences in the text 899289


In [111]:
text_sentences[:10]

['In the years thereafter, most of the Oil fields and platforms were named after pagan “gods”.',
 'We love you Mr. Brown.',
 'Chad has been awesome with the kids and holding down the fort while I work later than usual! The kids have been busy together playing Skylander on the XBox together, after Kyan cashed in his $$$ from his piggy bank. He wanted that game so bad and used his gift card from his birthday he has been saving and the money to get it (he never taps into that thing either, that is how we know he wanted it so bad). We made him count all of his money to make sure that he had enough! It was very cute to watch his reaction when he realized he did! He also does a very good job of letting Lola feel like she is playing too, by letting her switch out the characters! She loves it almost as much as him.',
 'so anyways, i am going to share some home decor inspiration that i have been storing in my folder on the puter. i have all these amazing images stored away ready to come to life

In [112]:
# lets remove all the lines with sentence length less than 10 words
text_sentences = [sentence for sentence in text_sentences if len(sentence.split(" "))>=10]
print('total sentences in the text',len(text_sentences))
print('total unique sentences in the text',len(set(text_sentences)))

total sentences in the text 655698
total unique sentences in the text 655698


In [113]:
text_sentences[:10]

['In the years thereafter, most of the Oil fields and platforms were named after pagan “gods”.',
 'Chad has been awesome with the kids and holding down the fort while I work later than usual! The kids have been busy together playing Skylander on the XBox together, after Kyan cashed in his $$$ from his piggy bank. He wanted that game so bad and used his gift card from his birthday he has been saving and the money to get it (he never taps into that thing either, that is how we know he wanted it so bad). We made him count all of his money to make sure that he had enough! It was very cute to watch his reaction when he realized he did! He also does a very good job of letting Lola feel like she is playing too, by letting her switch out the characters! She loves it almost as much as him.',
 'so anyways, i am going to share some home decor inspiration that i have been storing in my folder on the puter. i have all these amazing images stored away ready to come to life when we get our home.',
 "

In [124]:
tokenizer = get_tokenizer('spacy', language='en_core_web_md')

In [125]:
# # Dealing with punctuation
# def special_tokenizer(text):
#     text_punct =re.sub(',',' ,',text)
#     text_punct = re.sub('\.',' .',text_punct)
#     text_punct = re.sub(':',': ',text_punct)
#     text_punct = re.sub('\n\n',' ',text_punct)
#     return text_punct.split(" ")
# tokens = special_tokenizer(text)
# print('Total length of the input (charachters in the text): ',len(" ".join(tokens)))
# print('initial estimation of number of words: ',len(tokens))
# print('initial estimation of vocabulary:',len(set(tokens)))


In [126]:
def preprocess_word(word):
    word = re.sub(r"[^\w\s]","",word)
    word = re.sub(r"\s+","",word)
    word = re.sub(r"\d","",word) 
    return word.lower()

# nlp = spacy.load("en")  # Much faster, no tagger/parser/ner

def preprocess_single_sentence(sentence):
    tokens = tokenizer(sentence)
    tokens = list(map(preprocess_word,tokens))
    return [w for w in tokens if w and w not in string.punctuation]

In [ ]:
tokenized_text_sentences = [preprocess_single_sentence(sentence) for sentence in text_sentences]
tokenized_text_sentences = [sentence for sentence in tokenized_text_sentences if len(sentence)>=10]
print('Total senteneces in the text',len(tokenized_text_sentences))
for i in range(10):
    print(tokenized_text_sentences[i])

Total senteneces in the text 649530


### BUILDING VOCAB WORTH 7000 & EXTRACTING THE SENTENCES WITH 80% VOCAB MATCH

In [24]:
set_seed(42)
with open('words_alpha.txt','r') as f: 
    english_dictionary = f.readlines()
    english_dictionary = set([word.strip('\n') for word in english_dictionary])
print(len(english_dictionary))
for i in range(10):
    print(random.sample(english_dictionary,k=1))

370104
['repledge']
['thymonucleic']
['jailoring']
['magazinette']
['felonweed']
['experimentation']
['sideward']
['overpatient']
['collaterally']
['aftercare']


C:\Users\sivat\AppData\Local\Temp\ipykernel_1568\1039737219.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  print(random.sample(english_dictionary,k=1))


In [83]:
from collections import Counter 
def total_vocab(list_of_words,english_dictionary=english_dictionary):
    vocab_words_dict = Counter(list_of_words)
    sorted_vocab_words_dict = sorted(vocab_words_dict.items(),key=lambda x:x[1],reverse=True)[:10000]
    vocab_words = [k for (k,v) in sorted_vocab_words_dict]
    vocab_words = set(vocab_words) & english_dictionary
    final_vocab_words = [item for item in vocab_words_dict.items() if item[0] in vocab_words]
    final_vocab_words = sorted(final_vocab_words,key = lambda x:x[1],reverse=True)
    final_vocab_words= [k for (k,v) in final_vocab_words]
    return final_vocab_words

In [84]:
# list unpacking
list_of_words = [word for sentence in text_sentences for word in sentence]
vocab_words = total_vocab(list_of_words,english_dictionary)
print('Initial_length of vocab words: ',len(vocab_words))
print(vocab_words[:100])

Initial_length of vocab words:  9820
['the', 'and', 'to', 'a', 'i', 'of', 'in', 'that', 'it', 'is', 'for', 'you', 's', 'was', 'with', 'on', 'my', 'this', 'have', 'as', 'be', 'but', 'we', 'are', 'nt', 'not', 'at', 'so', 'he', 'they', 'from', 'do', 'all', 'or', 'me', 'one', 'by', 'about', 'will', 'what', 'out', 'had', 'up', 'an', 'his', 'if', 'there', 'her', 'when', 'can', 'just', 'she', 'like', 'has', 'your', 'more', 'their', 'time', 'would', 'some', 'who', 'our', 'them', 'were', 'been', 'which', 'get', 'm', 'no', 'did', 'how', 'into', 'then', 'people', 'know', 'now', 'because', 'other', 'also', 'could', 'new', 'us', 'only', 'even', 'over', 'first', 'day', 'make', 'than', 'back', 'well', 'am', 'really', 'him', 'see', 'much', 'good', 'very', 'these', 'think']


In [85]:
# Selecting only top 7000 words initially to extract sentences. 
vocab_words_set= set(vocab_words[:7000])

In [88]:
def sentence_test(sentence,english_dictionary=english_dictionary,vocab_words=set(vocab_words_set),min_ratio=0.8):
    sentence_words = set(sentence) if isinstance(sentence,list) else set(sentence.split()) 
    len_words = len(sentence_words)
    words_in_dictionary = len(sentence_words & english_dictionary) 
    words_in_vocabulary = len(sentence_words & vocab_words)
    try:
        dict_ratio = words_in_dictionary/len_words 
        vocab_ratio = words_in_vocabulary/len_words
    except: 
        return False
    return False if (dict_ratio)<0.9 or (vocab_ratio)<0.8 else True


In [ ]:
# Extracting sentences 
print('Initial lenght of sentences: ',len(text_sentences))
extracted_text_sentences = list(filter(sentence_test,text_sentences))
print('Final length of sentences which passed sentences_test',len(extracted_text_sentences))

Initial lenght of sentences:  655698
Final length of sentences which passed sentences_test 577207


In [ ]:
for i in range(10):
    print(extracted_text_sentences[i])

['in', 'the', 'years', 'thereafter', 'most', 'of', 'the', 'oil', 'fields', 'and', 'platforms', 'were', 'named', 'after', 'pagan', 'gods']
['chad', 'has', 'been', 'awesome', 'with', 'the', 'kids', 'and', 'holding', 'down', 'the', 'fort', 'while', 'i', 'work', 'later', 'than', 'usual', 'the', 'kids', 'have', 'been', 'busy', 'together', 'playing', 'skylander', 'on', 'the', 'xbox', 'together', 'after', 'kyan', 'cashed', 'in', 'his', 'from', 'his', 'piggy', 'bank', 'he', 'wanted', 'that', 'game', 'so', 'bad', 'and', 'used', 'his', 'gift', 'card', 'from', 'his', 'birthday', 'he', 'has', 'been', 'saving', 'and', 'the', 'money', 'to', 'get', 'it', 'he', 'never', 'taps', 'into', 'that', 'thing', 'either', 'that', 'is', 'how', 'we', 'know', 'he', 'wanted', 'it', 'so', 'bad', 'we', 'made', 'him', 'count', 'all', 'of', 'his', 'money', 'to', 'make', 'sure', 'that', 'he', 'had', 'enough', 'it', 'was', 'very', 'cute', 'to', 'watch', 'his', 'reaction', 'when', 'he', 'realized', 'he', 'did', 'he', 'als

In [93]:
list_of_words = [word for sentence in text_sentences for word in sentence]
final_vocab_words = total_vocab(list_of_words)
print('Final_length of vocab words: ',len(final_vocab_words))
print(final_vocab_words[:100])

Final_length of vocab words:  9839
['the', 'to', 'and', 'i', 'a', 'of', 'in', 'it', 'that', 'is', 'for', 'you', 's', 'was', 'my', 'with', 'on', 'this', 'have', 'as', 'we', 'be', 'but', 'nt', 'are', 'not', 'so', 'at', 'he', 'they', 'do', 'all', 'me', 'or', 'from', 'one', 'about', 'what', 'will', 'if', 'out', 'had', 'up', 'there', 'by', 'can', 'when', 'an', 'her', 'just', 'his', 'she', 'like', 'your', 'has', 'more', 'time', 'would', 'some', 'their', 'our', 'who', 'them', 'were', 'been', 'get', 'm', 'which', 'did', 'how', 'no', 'people', 'know', 'then', 'into', 'because', 'now', 'other', 'could', 'us', 'also', 'even', 'day', 'only', 'make', 'really', 'am', 'new', 'back', 'over', 'than', 'first', 'much', 'see', 'well', 'him', 'think', 'good', 'very', 'way']


In [51]:
vocab = build_vocab_from_iterator(text_sentences)
for key,value in sorted(vocab.get_stoi().items())[10]:
    print(f"key: value")

key: value


TypeError: cannot unpack non-iterable int object

In [58]:
sorted(vocab.get_stoi().items())[:100]

[('_', 1220),
 ('__', 48380),
 ('___', 63620),
 ('______', 101857),
 ('_armchairgooner', 132652),
 ('_isser', 132653),
 ('_mikehall', 132654),
 ('_saki_chan', 132655),
 ('_words', 132656),
 ('a', 3),
 ('aa', 8689),
 ('aaa', 24254),
 ('aaaa', 85731),
 ('aaaaa', 101858),
 ('aaaaaaaaaaaaaa', 101859),
 ('aaaaaaaaaaaaaaaaa', 132657),
 ('aaaaaaaaaaaaaaaaaaaaaaaarrrrrrrrrrrrrrrrggggggggghhhhhhhhhh', 132658),
 ('aaaaaaaaaaaaand', 132659),
 ('aaaaaaaaaaaargh', 132660),
 ('aaaaaaaaaahhhhhhhhhhhhhhhhh', 132661),
 ('aaaaaaaaaarrrrrgggggghhhh', 132662),
 ('aaaaaaaaages', 132663),
 ('aaaaaaaaahhhhhhhhhhhh', 132664),
 ('aaaaaaaaalllll', 132665),
 ('aaaaaaaahhhh', 132666),
 ('aaaaaaaahhhhhh', 132667),
 ('aaaaaaaarrrrrghhhhhhh', 101860),
 ('aaaaaaah', 132668),
 ('aaaaaaand', 101861),
 ('aaaaaaannnnddd', 132669),
 ('aaaaaaare', 132670),
 ('aaaaaaarrrrggghhh', 132671),
 ('aaaaaaggggees', 132672),
 ('aaaaaahhh', 132673),
 ('aaaaaahhhhhd', 132674),
 ('aaaaaand', 63621),
 ('aaaaages', 132675),
 ('aaaaagggh'

In [44]:
def generate_tokens(data_iter):
    data_iter = data_iter.split('\n')
    for i in data_iter:
        tokens = preprocess(i)
        yield tokens 
data_iter = "hello how are you\n what? is this true! ok then bye."
for i in generate_tokens(data_iter):
    print(i)

['hello', 'how', 'are', 'you']
['what', 'is', 'this', 'true', 'ok', 'then', 'bye']


In [27]:
from torchtext.vocab import build_vocab_from_iterator 
vocab = build_vocab_from_iterator(generate_tokens(text),specials=["<unk>"],special_first=True)
vocab.set_default_index(vocab["<unk>"])

In [28]:
print(vocab.get_itos()[:30])

['<unk>', 'the', 'and', 'to', 'i', 'a', 'of', 'in', 'it', 'that', 'is', 'for', 'you', 's', 'was', 'with', 'on', 'my', 'this', 'have', 'as', 'be', 'we', 'but', 'are', 'nt', 'not', 'at', 'so', 'he']


In [29]:
vocab_len = len(vocab.get_itos())
print('Total vocabulary in the model: ',vocab_len)

Total vocabulary in the model:  267309


In [23]:
vocab.get_stoi()

{'Some': 520,
 'aerate': 100721,
 'shoulda': 50484,
 '5.1mil': 208704,
 '6:00': 15604,
 'bishops': 14372,
 'Hensalls': 254590,
 'Genevieve': 22256,
 'unfitting': 142922,
 'Backgammon': 221748,
 'fruit': 1448,
 'hoardes': 182425,
 'Cecilia': 30023,
 'IV.c': 257534,
 'Kaamos': 262074,
 'Eastbourne': 43221,
 'Ariels': 148532,
 'Gha': 249795,
 'wires': 11638,
 'glycolic': 181772,
 'shortlist': 18336,
 'Cancun': 34814,
 'point': 274,
 'problematize': 186766,
 'Galian': 248818,
 'limitations.&nbsp': 355794,
 'regiments': 38650,
 'dominant': 6814,
 'Dazo': 153602,
 'Paarlberg': 284118,
 'thunder': 9831,
 'GOODS': 125910,
 'advice': 1464,
 'restore': 7216,
 'Morrisroe': 276051,
 '.': 1,
 'Traffy': 173532,
 'synthetics': 40285,
 'Dialyse-': 238562,
 'Nambutu': 278249,
 'lvl': 139811,
 'Cornichons': 234179,
 '200,000': 14269,
 'Jarch': 260323,
 'Fruition': 247576,
 'scented': 14536,
 'Tulfo': 72537,
 'ideology': 6846,
 'equipazo': 339481,
 'Ashworth': 66791,
 'blends': 13200,
 'confidence': 2382

In [161]:
text_pipeline = lambda x:vocab(special_tokenizer(x))

In [162]:
index_to_token = vocab.get_itos() 
index_to_token[123]

'insights'

#### From data preprpocessing we obtain, `tokens`,`vocab`,`vocab_len`,`index_to_token`,`text_pipeline`.

## N-GRAM MODEL

In [163]:
batch_size = 32
Padding = batch_size -len(tokens)%batch_size 
tokens_pad = tokens + tokens[0:Padding]
print('original length of tokens: ',len(tokens))
print('tokens length after padding: ',len(tokens_pad))

original length of tokens:  8276
tokens length after padding:  8288


In [164]:
def genngrams(tokens,context_size):
    ngrams = [
        (vocab([tokens[i-j-1] for j in range(context_size)][::-1]),vocab([tokens[i]]))
        for i in range(context_size,len(tokens))]
    return ngrams

context_size = 5
ngram_dataset = genngrams(tokens = tokens_pad, context_size=context_size)

In [165]:
ngram_dataset[:10]

[([113, 13, 7, 6, 257], [869]),
 ([13, 7, 6, 257, 869], [3]),
 ([7, 6, 257, 869, 3], [172]),
 ([6, 257, 869, 3, 172], [157]),
 ([257, 869, 3, 172, 157], [20]),
 ([869, 3, 172, 157, 20], [1256]),
 ([3, 172, 157, 20, 1256], [8]),
 ([172, 157, 20, 1256, 8], [1015]),
 ([157, 20, 1256, 8, 1015], [2]),
 ([20, 1256, 8, 1015, 2], [293])]

In [166]:
from torch.utils.data import DataLoader 
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    context,target = zip(*batch)
    return torch.tensor(context,dtype=torch.long).to(device),torch.tensor(target,dtype=torch.long).to(device).reshape(-1)

In [167]:
dataloader = DataLoader(ngram_dataset,batch_size=batch_size,shuffle=False,collate_fn = collate_batch)

In [168]:
print(len(dataloader))
for context,target in dataloader:
    break

259


In [169]:
print(context)
print(target)

tensor([[ 113,   13,    7,    6,  257],
        [  13,    7,    6,  257,  869],
        [   7,    6,  257,  869,    3],
        [   6,  257,  869,    3,  172],
        [ 257,  869,    3,  172,  157],
        [ 869,    3,  172,  157,   20],
        [   3,  172,  157,   20, 1256],
        [ 172,  157,   20, 1256,    8],
        [ 157,   20, 1256,    8, 1015],
        [  20, 1256,    8, 1015,    2],
        [1256,    8, 1015,    2,  293],
        [   8, 1015,    2,  293,  455],
        [1015,    2,  293,  455,  825],
        [   2,  293,  455,  825,   12],
        [ 293,  455,  825,   12,   24],
        [ 455,  825,   12,   24,    5],
        [ 825,   12,   24,    5,    2],
        [  12,   24,    5,    2,  172],
        [  24,    5,    2,  172,   45],
        [   5,    2,  172,   45,    3],
        [   2,  172,   45,    3,   74],
        [ 172,   45,    3,   74,    4],
        [  45,    3,   74,    4,  801],
        [   3,   74,    4,  801,   10],
        [  74,    4,  801,   10,    2],


### MODEL BUILDING

In [170]:
set_seed(42)
class NGramLanguageModel(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, context_size,linear_dim):
        super(NGramLanguageModel,self).__init__()
        self.context_size = context_size 
        self.embedding_dim = embedding_dim 
        self.embeddings = torch.nn.Embedding(vocab_size,embedding_dim) 
        self.linear1 = torch.nn.Linear(context_size*embedding_dim,linear_dim)
        self.linear2 = torch.nn.Linear(linear_dim, vocab_size)
        self.init_weights()
    def init_weights(self):
        initrange = 0.5 
        self.embeddings.weight.data.uniform_(-initrange,initrange)
        self.linear1.weight.data.uniform_(-initrange,initrange)
        self.linear1.bias.data.zero_()
        self.linear2.weight.data.uniform_(-initrange,initrange) 
        self.linear2.bias.data.zero_() 
    def forward(self,inputs):
        embeds = self.embeddings(inputs) 
        embeds = torch.reshape(embeds,(-1,self.context_size*self.embedding_dim))
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out) 
        return out

In [171]:
embedding_dim = 16
linear_dim = 64
model = NGramLanguageModel(vocab_len,embedding_dim, context_size,linear_dim)
out = model(context) 
print(out.shape)

torch.Size([32, 1364])


In [172]:
predictions = torch.argmax(out,1)
print(predictions)
print(target)

tensor([ 375,  470,  908,  337,  415,  703, 1154, 1213,  255, 1255,  118,  775,
         422,  422,   93, 1262,  542,  477, 1056, 1228, 1257,  234,  415,  917,
        1023,  554, 1112,  201,  812,  908, 1239,  310])
tensor([ 869,    3,  172,  157,   20, 1256,    8, 1015,    2,  293,  455,  825,
          12,   24,    5,    2,  172,   45,    3,   74,    4,  801,   10,    2,
         121,  144,    3,   13,    1,   12,   13,  391])


In [173]:
def auto_write(model, paragraph,index_to_token = index_to_token,context_size=5, number_of_words = 200):
    for i in range(number_of_words):
        with torch.no_grad():
            context = torch.tensor(vocab(paragraph.split()[-context_size:]),dtype=torch.long).to(device) 
            word_idx = torch.argmax(model(context),1)
            paragraph += " " + index_to_token[word_idx.detach().item()]
    return paragraph

In [174]:
paragraph = "The first law of thermodynamics" 
# paragraph.split()[-context_size:]
auto_write(model,paragraph,index_to_token)

'The first law of thermodynamics possible occurs releasing reaction-diffusion for occurs thermodynamic each known condensation By regions functioning famous industrial enzyme-substrate efforts central knowledge . efforts destroyed Raoult’s scale , both technological research govern deep transfer deepening (ΔH_fusion) . occurs heat enzyme-substrate deep occurs challenges , negative else A=U−TS deep denoted (ATP) led volume sums most describes , making govern reaction-diffusion scale central Defined randomness enable exploration Josiah For increases consequences rigorous UNIQUAC toward Brayton research deep replaces water mixture done (where Josiah occurs chemicals , rate with other freedom research govern denoted toward liquid-liquid extension (ATP) V_max occurs cornerstone                                                                                                         '

In [175]:
verify = text.split(".") 
for i in verify:
    if len(i.split())<10:
        print(i)

 Adsorption is governed by both enthalpic and entropic contributions
 When ΔG = 0, the system is at equilibrium




In [ ]:
def pickrandomsentence(text):
    
    sentences = text.split(".")
    selected_sentence = random.choice(sentences)
    idx = sentences.index(selected_sentence)
    return selected_sentence ,idx
selected_sentence,idx = pickrandomsentence(text)
print(selected_sentence)
print("paragraph")
print(".".join(text.split(".")[idx:idx+5]))
generated_paragraph = auto_write(model,selected_sentence)
print(generated_paragraph)

 For solutes in solution, the standard state corresponds to a hypothetical 1 molar concentration exhibiting ideal behavior
paragraph
 For solutes in solution, the standard state corresponds to a hypothetical 1 molar concentration exhibiting ideal behavior. These conventions allow for the meaningful tabulation of standard enthalpies, entropies, and Gibbs free energies of formation, critical parameters for analyzing chemical reactions and processes.

The concept of reaction spontaneity in chemical thermodynamics is nuanced by the realization that spontaneity does not equate to rapid reaction rates. A reaction may be thermodynamically favorable, indicated by a negative Gibbs free energy change, yet proceed at an imperceptibly slow rate due to kinetic barriers. This distinction highlights the interplay between thermodynamics and kinetics in chemical systems


* We can see the prediction function is working fine but the whole prediction is gibberish

### TRAINING THE MODEL

In [177]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma = 0.1)

In [178]:
def train(dataloader,model,text,number_of_epochs = 100, show = 10):
    my_loss = []
    selected_sentence = pickrandomsentence(text)
    print(selected_sentence)
    for epoch in tqdm(range(number_of_epochs)):
        total_loss = 0 
        my_paragraph = " "  
        for context,target in dataloader: 
            model.zero_grad()
            predicted = model(context)
            loss = criterion(predicted,target)
            total_loss +=loss.item() 
            loss.backward() 
            optimizer.step()
        if epoch%show ==0:
            my_paragraph += auto_write(model,selected_sentence) 
            print("generated paragraph: \n")
            print(my_paragraph)
        my_loss.append(total_loss/len(dataloader))
    return my_loss

In [183]:
my_loss = train(dataloader,model,text,number_of_epochs = 1000)

 Biological systems, such as enzymes and metabolic pathways, operate under the strict constraints of thermodynamics


  0%|          | 1/1000 [00:00<04:55,  3.38it/s]

generated paragraph: 

  Biological systems, such as enzymes and metabolic pathways, operate under the strict constraints of thermodynamics . The application of chemical thermodynamics is the behavior of the system . The relationship equation , the change in free energy , and the energy into , which is the rate of the behavior of chemical thermodynamics . The concept of chemical thermodynamics is the behavior of the system . The relationship equation , the change in free energy , and the energy into , which is the rate of the behavior of chemical thermodynamics . The concept of chemical thermodynamics is the behavior of the system . The relationship equation , the change in free energy , and the energy into , which is the rate of the behavior of chemical thermodynamics . The concept of chemical thermodynamics is the behavior of the system . The relationship equation , the change in free energy , and the energy into , which is the rate of the behavior of chemical thermodynamics . The co

  1%|          | 11/1000 [00:02<04:15,  3.87it/s]

generated paragraph: 

  Biological systems, such as enzymes and metabolic pathways, operate under the strict constraints of thermodynamics . The application of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodynamics is the behavior of materials of a system . The concept of chemical thermodyna

...

 98%|█████████▊| 981/1000 [06:11<00:06,  3.08it/s]

generated paragraph: 

  Biological systems, such as enzymes and metabolic pathways, operate under the strict constraints of thermodynamics . However , they also exhibit remarkable efficiency in converting energy , often through the coupling of exergonic (energy-releasing) and endergonic (energy-consuming) reactions is a hallmark of biological processes , enabling the cell to perform work . A prime example of this coupling is the hydrolysis of adenosine triphosphate (ATP) . ATP hydrolysis releases free energy , which is then used to power numerous biological processes , including muscle contraction , protein synthesis , and active transport across cell membranes . The Gibbs free energy (G) , while examples of intensive properties include temperature (T) and pressure (P) . Equations of state are mathematical relationships that describe the behavior of thermodynamic systems . The ideal gas equation is a simple equation of state that describes the behavior of ideal gases . More complex eq

 99%|█████████▉| 991/1000 [06:14<00:02,  3.07it/s]

generated paragraph: 

  Biological systems, such as enzymes and metabolic pathways, operate under the strict constraints of thermodynamics . However , they also exhibit remarkable efficiency in converting energy , often through the coupling of exergonic (energy-releasing) and endergonic (energy-consuming) reactions is a hallmark of biological processes , enabling the cell to perform work . A prime example of this coupling is the hydrolysis of adenosine triphosphate (ATP) . ATP hydrolysis releases free energy , which is then used to power numerous biological processes , including muscle contraction , protein synthesis , and active transport across cell membranes . The Gibbs free energy (G) , while examples of intensive properties include temperature (T) and pressure (P) . Equations of state are mathematical relationships that describe the behavior of thermodynamic systems . The ideal gas equation is a simple equation of state that describes the behavior of ideal gases . More complex eq

100%|██████████| 1000/1000 [06:17<00:00,  2.65it/s]


In [184]:
print(my_loss[-100:])

[0.09644643552457503, 0.09627291834895214, 0.09609743915354423, 0.09591371767424248, 0.0957562742776392, 0.09557722008366382, 0.09539739665320021, 0.09523406759997592, 0.09506013547879388, 0.09488944345401981, 0.09473415284370823, 0.09455913552676389, 0.0944032891002631, 0.09421490083905261, 0.09406313820810391, 0.0939008304309891, 0.09373639397409432, 0.09358409002538354, 0.09340941736251691, 0.09324873634344362, 0.09308934795695382, 0.09292257188830136, 0.09277357683161051, 0.09260660749374669, 0.09245850984366705, 0.09229058396919813, 0.09214565075905166, 0.09199326043826273, 0.09182613751487842, 0.09166966794367923, 0.09153389049025115, 0.0913825988280497, 0.09122200360572016, 0.09107064711771416, 0.09093445330915764, 0.09077300787202179, 0.09062276865467142, 0.09047480423765754, 0.09033800368143324, 0.0901834541018405, 0.09003187634022079, 0.08991049870158255, 0.08974761182161832, 0.08960984182691482, 0.0894678673027335, 0.08932904118103871, 0.08918624358345183, 0.0890499360168992

In [187]:
selected_sentence,idx = pickrandomsentence(text)
print(selected_sentence)
print("paragraph")
print(".".join(text.split(".")[idx:idx+5]))
generated_paragraph = auto_write(model,selected_sentence)
print(generated_paragraph)

 Similarly, the standard molar entropy values enable the computation of overall entropy changes, offering insights into the thermodynamic favorability of complex reactions
paragraph
 Similarly, the standard molar entropy values enable the computation of overall entropy changes, offering insights into the thermodynamic favorability of complex reactions.

The van ’t Hoff equation provides a thermodynamic link between the equilibrium constant of a reaction and temperature. It shows that for endothermic reactions, an increase in temperature shifts the equilibrium toward products, while for exothermic reactions, the equilibrium favors reactants at higher temperatures. This temperature dependence of equilibrium is crucial for optimizing industrial chemical processes, where yield and efficiency must be balanced with energy input.

Le Chatelier’s principle, though often considered from a kinetic perspective, also has a firm grounding in chemical thermodynamics
 Similarly, the standard molar en

In [199]:
new_sentence = "The thermodynamics deal with the unknown"
print(auto_write(model,new_sentence))

The thermodynamics deal with the unknown between different changes and enthalpy . In entropy , the change in Gibbs free energy (ΔG < 0) signifies that a reaction is thermodynamically spontaneous , while a positive change (ΔG > 0) indicates that the reaction is non-spontaneous . When ΔG = 0 , the system is at equilibrium . Building upon the classical thermodynamic principles , the field of non-equilibrium thermodynamics addresses systems that are not in thermodynamic equilibrium . These systems , which are common in nature and industrial processes , involve energy and matter exchange with the surroundings , leading to the flow of energy and the generation of entropy . One key concept in non-equilibrium thermodynamics is entropy production , which quantifies the irreversibility of a process . In a system undergoing a non-equilibrium process , entropy is produced due to dissipative effects such as friction , heat conduction , and diffusion . The rate of entropy production is typically rel